In [1]:
from selenium import webdriver # Selenium 모듈로 크롤링
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup # BeautifulSoup 모듈을 이용해 html 소스코드 객체구조 변환
from datetime import date # 날짜계산 모듈
from collections import Counter # 빈도수 모듈
from multiprocessing import Process, Queue # 멀티프로세싱
import pandas as pd # 데이터 후처리
import csv # csv 파일 저장
import time # sleep 함수

class instagram_Crawler:
    def follow_Crawling(self, path = '/Users/markmac/Downloads/chromedriver'): # 팔로우한 계정 크롤링
        """
            팔로우 계정 수집
            
            path: 크롬드라이버 설치 위치
        """
        self.path = path # Chrome Webdriver Path
        
        account_id = [] # 계정 ID 넣을 list
        
        driver = webdriver.Chrome(path) # Chrome webdriver path
        driver.get('https://www.instagram.com') # 사이트 주소
        time.sleep(1) # 1초 슬립
        
        driver.find_element_by_css_selector('p.izU2O > a').click() # 인스타그램 로그인 버튼 클릭
        time.sleep(1)
        
        driver.find_element_by_name('username').send_keys('hashtagcrawler@gmail.com') # 계정 ID 입력
        driver.find_element_by_name('password').send_keys('dkdlel12') # 계정 Password 입력
        
        driver.find_element_by_xpath("//button[@class = '_0mzm- sqdOP  L3NKy       ']").click() # 로그인 하기 버튼 클릭
        time.sleep(1)
        
        #driver.find_element_by_xpath("//button[@class = 'aOOlW  bIiDR  ']").click() # 알림 설정 버튼 클릭
        driver.find_element_by_xpath("//span[@class = 'glyphsSpriteUser__outline__24__grey_9 u-__7']").click() # 내 정보 클릭
        time.sleep(2)
        
        driver.find_elements_by_css_selector('a.-nal3 ')[1].click() # 팔로우 버튼 클릭
        time.sleep(1)
        
        scroll = driver.find_element_by_class_name('isgrP') # 스크롤 범위
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll) # 스크롤 맨 밑으로 내리기
        time.sleep(1)
        
        # 스크롤 절반 올리기
        jsScript = """
        function move_up(element) {
            element.scrollTop = element.scrollTop - 500;
        }

        function move_down(element) {
            console.log('Position before: ' + element.scrollTop);
            element.scrollTop = element.scrollTop + 1000;
            console.log('Position after: ' + element.scrollTop);
        }

        move_up(arguments[0]);
        """
        
        driver.execute_script(jsScript, scroll) # 스크롤 절반 올리기
        time.sleep(1)
        
        # 스크롤 다시 계속 내리기
        try:
            for i in range(10): # 스크롤 10번 내림
                driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll)
                time.sleep(2)
        except:
            print('error')
            
        time.sleep(1)
        
        html = driver.page_source # Html 소스코드 가져오기
        dom = BeautifulSoup(html, 'html.parser') # BeautifulSoup 모듈로 Html 객체구조 변환
            
        account_id_list = driver.find_elements_by_xpath("//a[@class = 'FPmhX notranslate _0imsa ']") # 계정 ID 경로
        
        account_id = [j.text for j in account_id_list] # 계정 ID 경로 텍스트로 리스트 삽입
            
        driver.close() # 브라우져 종료
            
        return account_id
    
    def instagram_Data(self, account_id, path = '/Users/markmac/Downloads/chromedriver'):
        self.account_id = account_id
        #self.output = output
        self.path = path
        
        account_Id = [] # 계정ID
        post_Count = [] # 게시물 수
        follower_Count = [] # 팔로워 수
        avg_Comment = [] # 평균댓글 수
        avg_Good = [] # 평균 좋아요 수
        comment_Count = [] # 총 댓글 수
        good_Count = [] # 총 좋아요 수
        last_Activity_Date = [] # 최근 활동 날짜
        first_Activity_Date = [] # 처음 활동 날짜
        active_Period = [] # 계정 활성 기간(일)
        hashtags = [] # 해시태그 소스코드
        words = [] # 해시태그 + 계정태그
        hashtags_data = [] # 순수 해시태그
        hashtags_Count = [] # 해시태그, 빈도수
        comment = 0
        good = 0
        
        for i in range(len(account_id)):
            driver = webdriver.Chrome(path)
            driver.get('https://www.instagram.com/' + account_id[i])

            time.sleep(1)
            html = driver.page_source
            dom = BeautifulSoup(html, 'html.parser')

            account_Id.append(dom.select('h1._7UhW9.fKFbl.yUEEX.KV-D4.fDxYl')[0].text) # 계정ID 저장
            post_Count.append(int(dom.select('span.g47SY')[0].text)) # 게시물 수 저장
            follower_Count.append(int(dom.select('span.g47SY')[1]['title'].replace(',', ''))) # 팔로워 수 저장

            driver.find_element_by_class_name('_9AhH0').click() # 첫번째 게시물 클릭

            while True:
                try:
                    time.sleep(1)
                    html = driver.page_source
                    dom = BeautifulSoup(html, 'html.parser')
                    
                    # 댓글 더 보기 버튼이 사라지면
                    if dom.select('li.lnrre') == []:
                        # 첫 게시물이 영상 게시물일때
                        if dom.select('div.Igw0E.IwRSH.eGOV_.ybXk5.vwCYk') == [] and comment == 0 and good == 0:
                            #account_info['마지막활동날짜'] = dom.select('time._1o9PC.Nzb55')[0]['title']
                            last_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10]) # 최근활동날짜 저장

                            last_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0]) # 최근활동년도
                            last_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1]) # 최근활동월
                            last_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2]) # 최근활동일

                            comment = len(dom.select('li.gElp9')) # 댓글 저장

                            hashtags += dom.select('div.C4VMK > span > a') # 해시태그 소스코드 저장

                            driver.find_element_by_class_name('vcOH2').click() # 조회수 클릭

                            time.sleep(1)
                            html = driver.page_source
                            dom = BeautifulSoup(html, 'html.parser')

                            good = int(dom.select('div.vJRqr > span')[0].text.replace(',', '')) # 좋아요 갯수 저장

                            driver.find_element_by_class_name('QhbhU').click() # 게시물 클릭 후 나감
                            time.sleep(1)
                            
                            # 다음 게시물이 없을 때
                            if len(dom.select('div.D1AKJ > a')) == 1 and dom.select('div.D1AKJ > a')[0].text == '이전':
                                time.sleep(1)
                                html = driver.page_source
                                dom = BeautifulSoup(html, 'html.parser')

                                #account_info['처음활동날짜'] = dom.select('time._1o9PC.Nzb55')[0]['title']
                                first_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10]) # 첫 활동 날짜 수집

                                first_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0]) # 첫 활동 년도
                                first_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1]) # 첫 활동 월
                                first_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2]) # 첫 활동 일

                                last = date(last_Year, last_Month, last_Day) # 최근활동날짜
                                first = date(first_Year, first_Month, first_Day) # 처음활동날짜

                                period = last - first # 활성기간
                                active_Period.append(period.days) # 활성기간 저장

                                comment_Count.append(comment) # 총 댓글 수 저장
                                good_Count.append(good) # 총 좋아요 수 저장
                                avg_Comment.append(round(comment / post_Count[0])) # 평균 댓글 수 저장
                                avg_Good.append(round(good / post_Count[0])) # 평균 좋아요 수 저장

                                words = [j.text for j in hashtags] # 해시태그 + 계정태그 저장

                                hashtags_data = [x for x in words if x[0] == '#'] # 순수 해시태그 추출

                                hashtags_Count.append(Counter(hashtags_data).most_common()[:1]) # 제일 빈도가 큰 해시태그 저장

                                driver.close() # 브라우저 닫기
                                break
                            
                            # 다음 게시물이 있으면
                            else:
                                # 다음 버튼 클릭
                                driver.find_element_by_xpath("//a[@class = 'HBoOv coreSpriteRightPaginationArrow']").click()
                                time.sleep(1)
                        
                        # 좋아요가 0인 영상 게시물일때
                        elif dom.select('div.Igw0E.IwRSH.eGOV_.ybXk5.vwCYk') == [] and good == 0 and comment != 0:
                            comment += len(dom.select('li.gElp9'))

                            hashtags += dom.select('div.C4VMK > span > a')

                            driver.find_element_by_class_name('vcOH2').click()

                            time.sleep(1)
                            html = driver.page_source
                            dom = BeautifulSoup(html, 'html.parser')

                            good = int(dom.select('div.vJRqr > span')[0].text.replace(',', ''))

                            driver.find_element_by_class_name('QhbhU').click()
                            time.sleep(1)

                            if len(dom.select('div.D1AKJ > a')) == 1 and dom.select('div.D1AKJ > a')[0].text == '이전':
                                time.sleep(1)
                                html = driver.page_source
                                dom = BeautifulSoup(html, 'html.parser')

                                #account_info['처음활동날짜'] = dom.select('time._1o9PC.Nzb55')[0]['title']
                                first_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10])

                                first_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0])
                                first_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1])
                                first_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2])

                                last = date(last_Year, last_Month, last_Day)
                                first = date(first_Year, first_Month, first_Day)

                                period = last - first
                                active_Period.append(period.days)

                                comment_Count.append(comment)
                                good_Count.append(good)
                                avg_Comment.append(round(comment / post_Count[0]))
                                avg_Good.append(round(good / post_Count[0]))

                                words = [j.text for j in hashtags]

                                hashtags_data = [x for x in words if x[0] == '#']

                                hashtags_Count.append(Counter(hashtags_data).most_common()[:1])

                                #total_data = {'계정ID':account_Id, '총게시물수':post_Count, '팔로워수':follower_Count, '평균댓글':avg_Comment, '평균좋아요':avg_Good, '총댓글수':comment_Count, '총좋아요수':good_Count, '최근게시물날짜':last_Activity_Date, '처음게시물날짜':first_Activity_Date, '활성기간':active_Period, '해시태그/빈도수':hashtags_Count}

                                #df = pd.DataFrame(total_data)
                                #df.to_csv('/Users/markmac/Downloads/instagram_accounts_data.csv')

                                driver.close()
                                break

                            else:
                                driver.find_element_by_xpath("//a[@class = 'HBoOv coreSpriteRightPaginationArrow']").click()
                                time.sleep(1)
                        
                        # 댓글수가 0일때 영상 게시물
                        elif dom.select('div.Igw0E.IwRSH.eGOV_.ybXk5.vwCYk') == [] and comment == 0 and good != 0:
                            comment = len(dom.select('li.gElp9'))

                            hashtags += dom.select('div.C4VMK > span > a')

                            driver.find_element_by_class_name('vcOH2').click()

                            time.sleep(1)
                            html = driver.page_source
                            dom = BeautifulSoup(html, 'html.parser')

                            good += int(dom.select('div.vJRqr > span')[0].text.replace(',', ''))

                            driver.find_element_by_class_name('QhbhU').click()
                            time.sleep(1)

                            if len(dom.select('div.D1AKJ > a')) == 1 and dom.select('div.D1AKJ > a')[0].text == '이전':
                                time.sleep(1)
                                html = driver.page_source
                                dom = BeautifulSoup(html, 'html.parser')

                                #account_info['처음활동날짜'] = dom.select('time._1o9PC.Nzb55')[0]['title']
                                first_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10])

                                first_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0])
                                first_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1])
                                first_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2])

                                last = date(last_Year, last_Month, last_Day)
                                first = date(first_Year, first_Month, first_Day)

                                period = last - first
                                active_Period.append(period.days)

                                comment_Count.append(comment)
                                good_Count.append(good)
                                avg_Comment.append(round(comment / post_Count[0]))
                                avg_Good.append(round(good / post_Count[0]))

                                words = [j.text for j in hashtags]

                                hashtags_data = [x for x in words if x[0] == '#']

                                hashtags_Count.append(Counter(hashtags_data).most_common()[:1])

                                #total_data = {'계정ID':account_Id, '총게시물수':post_Count, '팔로워수':follower_Count, '평균댓글':avg_Comment, '평균좋아요':avg_Good, '총댓글수':comment_Count, '총좋아요수':good_Count, '최근게시물날짜':last_Activity_Date, '처음게시물날짜':first_Activity_Date, '활성기간':active_Period, '해시태그/빈도수':hashtags_Count}

                                #df = pd.DataFrame(total_data)
                                #df.to_csv('/Users/markmac/Downloads/instagram_accounts_data.csv')

                                driver.close()
                                break

                            else:
                                driver.find_element_by_xpath("//a[@class = 'HBoOv coreSpriteRightPaginationArrow']").click()
                                time.sleep(1)
                        
                        # 댓글, 좋아요 수가 0이 아닐때 영상 게시물
                        elif dom.select('div.Igw0E.IwRSH.eGOV_.ybXk5.vwCYk') == [] and comment != 0 and good != 0:
                            comment += len(dom.select('li.gElp9'))

                            hashtags += dom.select('div.C4VMK > span > a')

                            driver.find_element_by_class_name('vcOH2').click()

                            time.sleep(1)
                            html = driver.page_source
                            dom = BeautifulSoup(html, 'html.parser')

                            good += int(dom.select('div.vJRqr > span')[0].text.replace(',', ''))

                            driver.find_element_by_class_name('QhbhU').click()
                            time.sleep(1)

                            if len(dom.select('div.D1AKJ > a')) == 1 and dom.select('div.D1AKJ > a')[0].text == '이전':
                                time.sleep(1)
                                html = driver.page_source
                                dom = BeautifulSoup(html, 'html.parser')

                                #account_info['처음활동날짜'] = dom.select('time._1o9PC.Nzb55')[0]['title']
                                first_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10])

                                first_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0])
                                first_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1])
                                first_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2])

                                last = date(last_Year, last_Month, last_Day)
                                first = date(first_Year, first_Month, first_Day)

                                period = last - first
                                active_Period.append(period.days)

                                comment_Count.append(comment)
                                good_Count.append(good)
                                avg_Comment.append(round(comment / post_Count[0]))
                                avg_Good.append(round(good / post_Count[0]))

                                words = [j.text for j in hashtags]

                                hashtags_data = [x for x in words if x[0] == '#']

                                hashtags_Count.append(Counter(hashtags_data).most_common()[:1])

                                #total_data = {'계정ID':account_Id, '총게시물수':post_Count, '팔로워수':follower_Count, '평균댓글':avg_Comment, '평균좋아요':avg_Good, '총댓글수':comment_Count, '총좋아요수':good_Count, '최근게시물날짜':last_Activity_Date, '처음게시물날짜':first_Activity_Date, '활성기간':active_Period, '해시태그/빈도수':hashtags_Count}

                                #df = pd.DataFrame(total_data)
                                #df.to_csv('/Users/markmac/Downloads/instagram_accounts_data.csv')

                                driver.close()
                                break

                            else:
                                driver.find_element_by_xpath("//a[@class = 'HBoOv coreSpriteRightPaginationArrow']").click()
                                time.sleep(1)
                        
                        # 첫 사진 게시물일때
                        elif dom.select('div.Igw0E.IwRSH.eGOV_.ybXk5.vwCYk') != [] and comment == 0 and good == 0:
                            #account_info['최근활동날짜'] = dom.select('time._1o9PC.Nzb55')[0]['title']
                            last_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10])

                            last_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0])
                            last_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1])
                            last_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2])

                            comment = len(dom.select('li.gElp9'))

                            hashtags += dom.select('div.C4VMK > span > a')

                            good = int(dom.select('button._0mzm-.sqdOP.yWX7d._8A5w5 > span')[0].text.replace(',', ''))

                            if len(dom.select('div.D1AKJ > a')) == 1 and dom.select('div.D1AKJ > a')[0].text == '이전':    
                                #account_info['처음활동날짜'] = dom.select('time._1o9PC.Nzb55')[0]['title']
                                first_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10])

                                first_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0])
                                first_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1])
                                first_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2])

                                last = date(last_Year, last_Month, last_Day)
                                first = date(first_Year, first_Month, first_Day)

                                period = last - first
                                active_Period.append(period.days)

                                comment_Count.append(comment)
                                good_Count.append(good)
                                avg_Comment.append(round(comment / post_Count[0]))
                                avg_Good.append(round(good / post_Count[0]))

                                words = [j.text for j in hashtags]

                                hashtags_data = [x for x in words if x[0] == '#']

                                hashtags_Count.append(Counter(hashtags_data).most_common()[:1])

                                #total_data = {'계정ID':account_Id, '총게시물수':post_Count, '팔로워수':follower_Count, '평균댓글':avg_Comment, '평균좋아요':avg_Good, '총댓글수':comment_Count, '총좋아요수':good_Count, '최근게시물날짜':last_Activity_Date, '처음게시물날짜':first_Activity_Date, '활성기간':active_Period, '해시태그/빈도수':hashtags_Count}

                                #df = pd.DataFrame(total_data)
                                #df.to_csv('/Users/markmac/Downloads/instagram_accounts_data.csv')

                                driver.close()
                                break

                            else:
                                driver.find_element_by_xpath("//a[@class = 'HBoOv coreSpriteRightPaginationArrow']").click()
                                time.sleep(1)
                        
                        # 댓글 수가 0일때 사진 게시물
                        elif dom.select('div.Igw0E.IwRSH.eGOV_.ybXk5.vwCYk') != [] and comment == 0 and good != 0:
                            comment = len(dom.select('li.gElp9'))

                            hashtags += dom.select('div.C4VMK > span > a')

                            good += int(dom.select('button._0mzm-.sqdOP.yWX7d._8A5w5 > span')[0].text.replace(',', ''))

                            if len(dom.select('div.D1AKJ > a')) == 1 and dom.select('div.D1AKJ > a')[0].text == '이전':    
                                #account_info['처음활동날짜'] = dom.select('time._1o9PC.Nzb55')[0]['title']
                                first_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10])

                                first_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0])
                                first_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1])
                                first_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2])

                                last = date(last_Year, last_Month, last_Day)
                                first = date(first_Year, first_Month, first_Day)

                                period = last - first
                                active_Period.append(period.days)

                                comment_Count.append(comment)
                                good_Count.append(good)
                                avg_Comment.append(round(comment / post_Count[0]))
                                avg_Good.append(round(good / post_Count[0]))

                                words = [j.text for j in hashtags]

                                hashtags_data = [x for x in words if x[0] == '#']

                                hashtags_Count.append(Counter(hashtags_data).most_common()[:1])

                                #total_data = {'계정ID':account_Id, '총게시물수':post_Count, '팔로워수':follower_Count, '평균댓글':avg_Comment, '평균좋아요':avg_Good, '총댓글수':comment_Count, '총좋아요수':good_Count, '최근게시물날짜':last_Activity_Date, '처음게시물날짜':first_Activity_Date, '활성기간':active_Period, '해시태그/빈도수':hashtags_Count}

                                #df = pd.DataFrame(total_data)
                                #df.to_csv('/Users/markmac/Downloads/instagram_accounts_data.csv')

                                driver.close()
                                break

                            else:
                                driver.find_element_by_xpath("//a[@class = 'HBoOv coreSpriteRightPaginationArrow']").click()
                                time.sleep(1)
                        
                        # 좋아요 수가 0일때 사진게시물
                        elif dom.select('div.Igw0E.IwRSH.eGOV_.ybXk5.vwCYk') != [] and good == 0 and comment != 0:
                            comment += len(dom.select('li.gElp9'))

                            hashtags += dom.select('div.C4VMK > span > a')

                            good = int(dom.select('button._0mzm-.sqdOP.yWX7d._8A5w5 > span')[0].text.replace(',', ''))

                            if len(dom.select('div.D1AKJ > a')) == 1 and dom.select('div.D1AKJ > a')[0].text == '이전':    
                                #account_info['처음활동날짜'] = dom.select('time._1o9PC.Nzb55')[0]['title']
                                first_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10])

                                first_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0])
                                first_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1])
                                first_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2])

                                last = date(last_Year, last_Month, last_Day)
                                first = date(first_Year, first_Month, first_Day)

                                period = last - first
                                active_Period.append(period.days)

                                comment_Count.append(comment)
                                good_Count.append(good)
                                avg_Comment.append(round(comment / post_Count[0]))
                                avg_Good.append(round(good / post_Count[0]))

                                words = [j.text for j in hashtags]

                                hashtags_data = [x for x in words if x[0] == '#']

                                hashtags_Count.append(Counter(hashtags_data).most_common()[:1])

                                #total_data = {'계정ID':account_Id, '총게시물수':post_Count, '팔로워수':follower_Count, '평균댓글':avg_Comment, '평균좋아요':avg_Good, '총댓글수':comment_Count, '총좋아요수':good_Count, '최근게시물날짜':last_Activity_Date, '처음게시물날짜':first_Activity_Date, '활성기간':active_Period, '해시태그/빈도수':hashtags_Count}

                                #df = pd.DataFrame(total_data)
                                #df.to_csv('/Users/markmac/Downloads/instagram_accounts_data.csv')

                                driver.close()
                                break

                            else:
                                driver.find_element_by_xpath("//a[@class = 'HBoOv coreSpriteRightPaginationArrow']").click()
                                time.sleep(1)
                        
                        # 둘다 0일때 사진 게시물
                        else:
                            comment += len(dom.select('li.gElp9'))

                            hashtags += dom.select('div.C4VMK > span > a')

                            good += int(dom.select('button._0mzm-.sqdOP.yWX7d._8A5w5 > span')[0].text.replace(',', ''))

                            if len(dom.select('div.D1AKJ > a')) == 1 and dom.select('div.D1AKJ > a')[0].text == '이전':    
                                #account_info['처음활동날짜'] = dom.select('time._1o9PC.Nzb55')[0]['title']
                                first_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10])

                                first_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0])
                                first_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1])
                                first_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2])

                                last = date(last_Year, last_Month, last_Day)
                                first = date(first_Year, first_Month, first_Day)

                                period = last - first
                                active_Period.append(period.days)

                                comment_Count.append(comment)
                                good_Count.append(good)
                                avg_Comment.append(round(comment / post_Count[0]))
                                avg_Good.append(round(good / post_Count[0]))

                                words = [j.text for j in hashtags]

                                hashtags_data = [x for x in words if x[0] == '#']

                                hashtags_Count.append(Counter(hashtags_data).most_common()[:1])

                                #total_data = {'계정ID':account_Id, '총게시물수':post_Count, '팔로워수':follower_Count, '평균댓글':avg_Comment, '평균좋아요':avg_Good, '총댓글수':comment_Count, '총좋아요수':good_Count, '최근게시물날짜':last_Activity_Date, '처음게시물날짜':first_Activity_Date, '활성기간':active_Period, '해시태그/빈도수':hashtags_Count}

                                #df = pd.DataFrame(total_data)
                                #df.to_csv('/Users/markmac/Downloads/instagram_accounts_data.csv')

                                driver.close()
                                break

                            else:
                                driver.find_element_by_xpath("//a[@class = 'HBoOv coreSpriteRightPaginationArrow']").click()
                                time.sleep(1)
                    
                    # 댓글 더보기 버튼이 있으면
                    else:
                        #more_Comment_Wait = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'button.Z4IfV._0mzm-.sqdOP.yWX7d')))
                        #more_Comment_Wait.click()
                        
                        # 댓글 더 보기 버튼 클릭
                        driver.find_element_by_xpath("//button[@class = 'Z4IfV _0mzm- sqdOP yWX7d        ']").click()
                        time.sleep(1)
                except:
                    break
        # dict 형식으로 모든 데이터 저장
        total_data = {'계정ID':account_Id, '총게시물수':post_Count, '팔로워수':follower_Count, '평균댓글':avg_Comment, '평균좋아요':avg_Good, '총댓글수':comment_Count, '총좋아요수':good_Count, '최근게시물날짜':last_Activity_Date, '처음게시물날짜':first_Activity_Date, '활성기간':active_Period, '해시태그/빈도수':hashtags_Count}
        
        #output.put(total_data)
        
        df = pd.DataFrame(total_data) # pandas dataframe 형식으로 저장
        df.to_csv('/Users/markmac/Downloads/instagram_accounts_data.csv') # csv 파일로 저장
        
    def save_csv(self, total_data):
        self.total_data = total_data
        
        df = pd.DataFrame(total_data)
        df.to_csv('/Users/markmac/Downloads/instagram_accounts_data.csv')

In [31]:
a = [23, 12]
b = [235, 123]
c = [234, 1234]
d = ('sdfg', 2355), ('xcsdf', 867)

In [32]:
qq = {'dsf':a, 'xcv':b, 'sdfg':c, 'sdfsd':d}

In [33]:
qq

{'dsf': [23, 12],
 'xcv': [235, 123],
 'sdfg': [234, 1234],
 'sdfsd': (('sdfg', 2355), ('xcsdf', 867))}

In [34]:
df = pd.DataFrame(qq)

In [35]:
df

,dsf,xcv,sdfg,sdfsd
0,23,235,234,"(sdfg, 2355)"
1,12,123,1234,"(xcsdf, 867)"


In [2]:
instagram = instagram_Crawler()

In [3]:
accounts_Id_Lists = instagram.follow_Crawling()

In [4]:
accounts_Id_Lists

['jayflowsik',
 'killagramz3xl',
 'chachamalone',
 'boycold_',
 'uglyduck062',
 'wavycolde',
 'woogie_park',
 'longlivesmdc',
 'anythinggoes85',
 'coogie0123',
 'hwimmm',
 'yunbnotlilb',
 'supergreathurricane',
 'drunkentigerjk',
 'g2slife',
 'gaekogeem',
 'nucksal',
 'penomadeincorea',
 'keemhongwu',
 'huckleberryp84',
 'deepflow39',
 'superstarjimmyfuckedup',
 'originalgimchi',
 'hiolnl',
 'dprlive',
 'crush9244',
 'satgotloco',
 'junoflo',
 'woodie_gochild',
 'younghotyellow94',
 'code_kunst',
 'bewhy.meshasoulja',
 'nasungcityboy',
 'wejustmusic',
 'thisloop',
 'jparkitrighthere',
 'hashblanccoa',
 'callmegray',
 'aomgofficial',
 'ph1boyyy',
 'paloaltongue',
 'woozico0914',
 'thequiett',
 'munchinthepool',
 'dok2gonzo',
 'noahmik',
 'realllllmino',
 'kiff_websterb',
 'changmo_',
 'itsjustswings',
 'kidcozyboy',
 'noelcozyboy',
 'yohanheaven',
 'osshun_gum',
 'jvckiwai',
 'hankyungsoo',
 'giriboy91']

In [6]:
len(accounts_Id_Lists)

57

In [8]:
account_id_lists_1 = []
account_id_lists_2 = []
account_id_lists_3 = []
account_id_lists_4 = []
account_id_lists_5 = []
account_id_lists_6 = []

# multiprocessing 모듈을 사용하기 위해 수집한 계정데이터 분리
account_id_lists_1 += accounts_Id_Lists[:9]
account_id_lists_2 += accounts_Id_Lists[9:18]
account_id_lists_3 += accounts_Id_Lists[18:28]
account_id_lists_4 += accounts_Id_Lists[28:38]
account_id_lists_5 += accounts_Id_Lists[38:48]
account_id_lists_6 += accounts_Id_Lists[48:57]

In [9]:
account_id_lists_1

['jayflowsik',
 'killagramz3xl',
 'chachamalone',
 'boycold_',
 'uglyduck062',
 'wavycolde',
 'woogie_park',
 'longlivesmdc',
 'anythinggoes85']

In [10]:
account_id_lists_2

['coogie0123',
 'hwimmm',
 'yunbnotlilb',
 'supergreathurricane',
 'drunkentigerjk',
 'g2slife',
 'gaekogeem',
 'nucksal',
 'penomadeincorea']

In [11]:
account_id_lists_3

['keemhongwu',
 'huckleberryp84',
 'deepflow39',
 'superstarjimmyfuckedup',
 'originalgimchi',
 'hiolnl',
 'dprlive',
 'crush9244',
 'satgotloco',
 'junoflo']

In [12]:
account_id_lists_4

['woodie_gochild',
 'younghotyellow94',
 'code_kunst',
 'bewhy.meshasoulja',
 'nasungcityboy',
 'wejustmusic',
 'thisloop',
 'jparkitrighthere',
 'hashblanccoa',
 'callmegray']

In [13]:
account_id_lists_5

['aomgofficial',
 'ph1boyyy',
 'paloaltongue',
 'woozico0914',
 'thequiett',
 'munchinthepool',
 'dok2gonzo',
 'noahmik',
 'realllllmino',
 'kiff_websterb']

In [14]:
account_id_lists_6

['changmo_',
 'itsjustswings',
 'kidcozyboy',
 'noelcozyboy',
 'yohanheaven',
 'osshun_gum',
 'jvckiwai',
 'hankyungsoo',
 'giriboy91']

In [20]:
# 6개의 브라우저를 생성해서 해시태그 데이터 수집
if __name__ == '__main__':
    output1 = Queue()
    output2 = Queue()
    output3 = Queue()
    output4 = Queue()
    output5 = Queue()
    output6 = Queue()
    
    temp = []
    procs = []
    procs.append(Process(target = instagram.instagram_Data, args = (account_id_lists_1, output1)))
    procs.append(Process(target = instagram.instagram_Data, args = (account_id_lists_2, output2)))
    procs.append(Process(target = instagram.instagram_Data, args = (account_id_lists_3, output3)))
    procs.append(Process(target = instagram.instagram_Data, args = (account_id_lists_4, output4)))
    procs.append(Process(target = instagram.instagram_Data, args = (account_id_lists_5, output5)))
    procs.append(Process(target = instagram.instagram_Data, args = (account_id_lists_6, output6)))
    
    for p in procs:
        p.start()
        
    temp += output1.get()
    temp += output2.get()
    temp += output3.get()
    temp += output4.get()
    temp += output5.get()
    temp += output6.get()
    
    for p in procs:
        p.join()

Process Process-41:
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-1-b58268204af9>", line 118, in instagram_Data
    post_Count.append(int(dom.select('span.g47SY')[0].text))
ValueError: invalid literal for int() with base 10: '1,479'
Process Process-39:
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-1-b58268204af9>", line 118, in instagram_Data
    post_Count.append(int(dom.select('span.g47SY')[0].text))
ValueError: invalid literal for int() with base 10: '2,617'
Process Process-38:
Process Process-42:


KeyboardInterrupt: 

Process Process-37:
Process Process-40:
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-1-b58268204af9>", line 110, in instagram_Data
    driver = webdriver.Chrome(path) # chromedriver 실행파일 저장위치
  File "/anaconda3/lib/python3.7/site-packages/selenium/webdriver/chrome/webdriver.py", line 73, in __init__
    self.service.start()
  File "/anaconda3/lib/python3.7/site-packages/selenium/webdriver/common/service.py", line 102, in start
    time.sleep(1)
KeyboardInterrupt


In [15]:
temp

NameError: name 'temp' is not defined

In [36]:
#aa = ['thisloop', 'hiolnl']
aa = ['hiolnl']

In [37]:
instagram.instagram_Data(aa)

ValueError: arrays must all be same length

In [29]:
df = pd.DataFrame({'sd':['sda', 'sdfs']})

In [30]:
df

,sd
0,sda
1,sdfs


In [39]:
account_Id = [] # 계정ID
post_Count = [] # 게시물 수
follower_Count = [] # 팔로워 수
avg_Comment = [] # 평균댓글 수
avg_Good = [] # 평균 좋아요 수
comment_Count = [] # 총 댓글 수
good_Count = [] # 총 좋아요 수
last_Activity_Date = [] # 최근 활동 날짜
first_Activity_Date = [] # 처음 활동 날짜
active_Period = [] # 계정 활성 기간(일)
hashtags = [] # 해시태그 소스코드
words = [] # 해시태그 + 계정태그
hashtags_data = [] # 순수 해시태그
hashtags_Count = [] # 해시태그, 빈도수
comment = 0
good = 0

driver = webdriver.Chrome('/Users/markmac/Downloads/chromedriver')
driver.get('https://www.instagram.com/hiolnl')

time.sleep(1)
html = driver.page_source
dom = BeautifulSoup(html, 'html.parser')

account_Id.append(dom.select('h1._7UhW9.fKFbl.yUEEX.KV-D4.fDxYl')[0].text) # 계정ID 저장
post_Count.append(int(dom.select('span.g47SY')[0].text)) # 게시물 수 저장
follower_Count.append(int(dom.select('span.g47SY')[1]['title'].replace(',', ''))) # 팔로워 수 저장

driver.find_element_by_class_name('_9AhH0').click() # 첫번째 게시물 클릭

while True:
    try:
        time.sleep(1)
        html = driver.page_source
        dom = BeautifulSoup(html, 'html.parser')

        # 댓글 더 보기 버튼이 사라지면
        if dom.select('li.lnrre') == []:
            # 첫 게시물이 영상 게시물일때
            if dom.select('div.Igw0E.IwRSH.eGOV_.ybXk5.vwCYk') == [] and comment == 0 and good == 0:
                #account_info['마지막활동날짜'] = dom.select('time._1o9PC.Nzb55')[0]['title']
                last_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10]) # 최근활동날짜 저장

                last_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0]) # 최근활동년도
                last_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1]) # 최근활동월
                last_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2]) # 최근활동일

                comment = len(dom.select('li.gElp9')) # 댓글 저장

                hashtags += dom.select('div.C4VMK > span > a') # 해시태그 소스코드 저장

                driver.find_element_by_class_name('vcOH2').click() # 조회수 클릭

                time.sleep(1)
                html = driver.page_source
                dom = BeautifulSoup(html, 'html.parser')

                good = int(dom.select('div.vJRqr > span')[0].text.replace(',', '')) # 좋아요 갯수 저장

                driver.find_element_by_class_name('QhbhU').click() # 게시물 클릭 후 나감
                time.sleep(1)

                # 다음 게시물이 없을 때
                if len(dom.select('div.D1AKJ > a')) == 1 and dom.select('div.D1AKJ > a')[0].text == '이전':
                    time.sleep(1)
                    html = driver.page_source
                    dom = BeautifulSoup(html, 'html.parser')

                    #account_info['처음활동날짜'] = dom.select('time._1o9PC.Nzb55')[0]['title']
                    first_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10]) # 첫 활동 날짜 수집

                    first_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0]) # 첫 활동 년도
                    first_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1]) # 첫 활동 월
                    first_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2]) # 첫 활동 일

                    last = date(last_Year, last_Month, last_Day) # 최근활동날짜
                    first = date(first_Year, first_Month, first_Day) # 처음활동날짜

                    period = last - first # 활성기간
                    active_Period.append(period.days) # 활성기간 저장

                    comment_Count.append(comment) # 총 댓글 수 저장
                    good_Count.append(good) # 총 좋아요 수 저장
                    avg_Comment.append(round(comment / post_Count[0])) # 평균 댓글 수 저장
                    avg_Good.append(round(good / post_Count[0])) # 평균 좋아요 수 저장

                    words = [j.text for j in hashtags] # 해시태그 + 계정태그 저장

                    hashtags_data = [x for x in words if x[0] == '#'] # 순수 해시태그 추출

                    hashtags_Count.append(Counter(hashtags_data).most_common()[:1]) # 제일 빈도가 큰 해시태그 저장

                    driver.close() # 브라우저 닫기
                    break

                # 다음 게시물이 있으면
                else:
                    # 다음 버튼 클릭
                    driver.find_element_by_xpath("//a[@class = 'HBoOv coreSpriteRightPaginationArrow']").click()
                    time.sleep(1)

            # 좋아요가 0인 영상 게시물일때
            elif dom.select('div.Igw0E.IwRSH.eGOV_.ybXk5.vwCYk') == [] and good == 0 and comment != 0:
                comment += len(dom.select('li.gElp9'))

                hashtags += dom.select('div.C4VMK > span > a')

                driver.find_element_by_class_name('vcOH2').click()

                time.sleep(1)
                html = driver.page_source
                dom = BeautifulSoup(html, 'html.parser')

                good = int(dom.select('div.vJRqr > span')[0].text.replace(',', ''))

                driver.find_element_by_class_name('QhbhU').click()
                time.sleep(1)

                if len(dom.select('div.D1AKJ > a')) == 1 and dom.select('div.D1AKJ > a')[0].text == '이전':
                    time.sleep(1)
                    html = driver.page_source
                    dom = BeautifulSoup(html, 'html.parser')

                    #account_info['처음활동날짜'] = dom.select('time._1o9PC.Nzb55')[0]['title']
                    first_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10])

                    first_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0])
                    first_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1])
                    first_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2])

                    last = date(last_Year, last_Month, last_Day)
                    first = date(first_Year, first_Month, first_Day)

                    period = last - first
                    active_Period.append(period.days)

                    comment_Count.append(comment)
                    good_Count.append(good)
                    avg_Comment.append(round(comment / post_Count[0]))
                    avg_Good.append(round(good / post_Count[0]))

                    words = [j.text for j in hashtags]

                    hashtags_data = [x for x in words if x[0] == '#']

                    hashtags_Count.append(Counter(hashtags_data).most_common()[:1])

                    #total_data = {'계정ID':account_Id, '총게시물수':post_Count, '팔로워수':follower_Count, '평균댓글':avg_Comment, '평균좋아요':avg_Good, '총댓글수':comment_Count, '총좋아요수':good_Count, '최근게시물날짜':last_Activity_Date, '처음게시물날짜':first_Activity_Date, '활성기간':active_Period, '해시태그/빈도수':hashtags_Count}

                    #df = pd.DataFrame(total_data)
                    #df.to_csv('/Users/markmac/Downloads/instagram_accounts_data.csv')

                    driver.close()
                    break

                else:
                    driver.find_element_by_xpath("//a[@class = 'HBoOv coreSpriteRightPaginationArrow']").click()
                    time.sleep(1)

            # 댓글수가 0일때 영상 게시물
            elif dom.select('div.Igw0E.IwRSH.eGOV_.ybXk5.vwCYk') == [] and comment == 0 and good != 0:
                comment = len(dom.select('li.gElp9'))

                hashtags += dom.select('div.C4VMK > span > a')

                driver.find_element_by_class_name('vcOH2').click()

                time.sleep(1)
                html = driver.page_source
                dom = BeautifulSoup(html, 'html.parser')

                good += int(dom.select('div.vJRqr > span')[0].text.replace(',', ''))

                driver.find_element_by_class_name('QhbhU').click()
                time.sleep(1)

                if len(dom.select('div.D1AKJ > a')) == 1 and dom.select('div.D1AKJ > a')[0].text == '이전':
                    time.sleep(1)
                    html = driver.page_source
                    dom = BeautifulSoup(html, 'html.parser')

                    #account_info['처음활동날짜'] = dom.select('time._1o9PC.Nzb55')[0]['title']
                    first_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10])

                    first_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0])
                    first_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1])
                    first_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2])

                    last = date(last_Year, last_Month, last_Day)
                    first = date(first_Year, first_Month, first_Day)

                    period = last - first
                    active_Period.append(period.days)

                    comment_Count.append(comment)
                    good_Count.append(good)
                    avg_Comment.append(round(comment / post_Count[0]))
                    avg_Good.append(round(good / post_Count[0]))

                    words = [j.text for j in hashtags]

                    hashtags_data = [x for x in words if x[0] == '#']

                    hashtags_Count.append(Counter(hashtags_data).most_common()[:1])

                    #total_data = {'계정ID':account_Id, '총게시물수':post_Count, '팔로워수':follower_Count, '평균댓글':avg_Comment, '평균좋아요':avg_Good, '총댓글수':comment_Count, '총좋아요수':good_Count, '최근게시물날짜':last_Activity_Date, '처음게시물날짜':first_Activity_Date, '활성기간':active_Period, '해시태그/빈도수':hashtags_Count}

                    #df = pd.DataFrame(total_data)
                    #df.to_csv('/Users/markmac/Downloads/instagram_accounts_data.csv')

                    driver.close()
                    break

                else:
                    driver.find_element_by_xpath("//a[@class = 'HBoOv coreSpriteRightPaginationArrow']").click()
                    time.sleep(1)

            # 댓글, 좋아요 수가 0이 아닐때 영상 게시물
            elif dom.select('div.Igw0E.IwRSH.eGOV_.ybXk5.vwCYk') == [] and comment != 0 and good != 0:
                comment += len(dom.select('li.gElp9'))

                hashtags += dom.select('div.C4VMK > span > a')

                driver.find_element_by_class_name('vcOH2').click()

                time.sleep(1)
                html = driver.page_source
                dom = BeautifulSoup(html, 'html.parser')

                good += int(dom.select('div.vJRqr > span')[0].text.replace(',', ''))

                driver.find_element_by_class_name('QhbhU').click()
                time.sleep(1)

                if len(dom.select('div.D1AKJ > a')) == 1 and dom.select('div.D1AKJ > a')[0].text == '이전':
                    time.sleep(1)
                    html = driver.page_source
                    dom = BeautifulSoup(html, 'html.parser')

                    #account_info['처음활동날짜'] = dom.select('time._1o9PC.Nzb55')[0]['title']
                    first_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10])

                    first_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0])
                    first_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1])
                    first_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2])

                    last = date(last_Year, last_Month, last_Day)
                    first = date(first_Year, first_Month, first_Day)

                    period = last - first
                    active_Period.append(period.days)

                    comment_Count.append(comment)
                    good_Count.append(good)
                    avg_Comment.append(round(comment / post_Count[0]))
                    avg_Good.append(round(good / post_Count[0]))

                    words = [j.text for j in hashtags]

                    hashtags_data = [x for x in words if x[0] == '#']

                    hashtags_Count.append(Counter(hashtags_data).most_common()[:1])

                    #total_data = {'계정ID':account_Id, '총게시물수':post_Count, '팔로워수':follower_Count, '평균댓글':avg_Comment, '평균좋아요':avg_Good, '총댓글수':comment_Count, '총좋아요수':good_Count, '최근게시물날짜':last_Activity_Date, '처음게시물날짜':first_Activity_Date, '활성기간':active_Period, '해시태그/빈도수':hashtags_Count}

                    #df = pd.DataFrame(total_data)
                    #df.to_csv('/Users/markmac/Downloads/instagram_accounts_data.csv')

                    driver.close()
                    break

                else:
                    driver.find_element_by_xpath("//a[@class = 'HBoOv coreSpriteRightPaginationArrow']").click()
                    time.sleep(1)

            # 첫 사진 게시물일때
            elif dom.select('div.Igw0E.IwRSH.eGOV_.ybXk5.vwCYk') != [] and comment == 0 and good == 0:
                #account_info['최근활동날짜'] = dom.select('time._1o9PC.Nzb55')[0]['title']
                last_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10])

                last_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0])
                last_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1])
                last_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2])

                comment = len(dom.select('li.gElp9'))

                hashtags += dom.select('div.C4VMK > span > a')

                good = int(dom.select('button._0mzm-.sqdOP.yWX7d._8A5w5 > span')[0].text.replace(',', ''))

                if len(dom.select('div.D1AKJ > a')) == 1 and dom.select('div.D1AKJ > a')[0].text == '이전':    
                    #account_info['처음활동날짜'] = dom.select('time._1o9PC.Nzb55')[0]['title']
                    first_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10])

                    first_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0])
                    first_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1])
                    first_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2])

                    last = date(last_Year, last_Month, last_Day)
                    first = date(first_Year, first_Month, first_Day)

                    period = last - first
                    active_Period.append(period.days)

                    comment_Count.append(comment)
                    good_Count.append(good)
                    avg_Comment.append(round(comment / post_Count[0]))
                    avg_Good.append(round(good / post_Count[0]))

                    words = [j.text for j in hashtags]

                    hashtags_data = [x for x in words if x[0] == '#']

                    hashtags_Count.append(Counter(hashtags_data).most_common()[:1])

                    #total_data = {'계정ID':account_Id, '총게시물수':post_Count, '팔로워수':follower_Count, '평균댓글':avg_Comment, '평균좋아요':avg_Good, '총댓글수':comment_Count, '총좋아요수':good_Count, '최근게시물날짜':last_Activity_Date, '처음게시물날짜':first_Activity_Date, '활성기간':active_Period, '해시태그/빈도수':hashtags_Count}

                    #df = pd.DataFrame(total_data)
                    #df.to_csv('/Users/markmac/Downloads/instagram_accounts_data.csv')

                    driver.close()
                    break

                else:
                    driver.find_element_by_xpath("//a[@class = 'HBoOv coreSpriteRightPaginationArrow']").click()
                    time.sleep(1)

            # 댓글 수가 0일때 사진 게시물
            elif dom.select('div.Igw0E.IwRSH.eGOV_.ybXk5.vwCYk') != [] and comment == 0 and good != 0:
                comment = len(dom.select('li.gElp9'))

                hashtags += dom.select('div.C4VMK > span > a')

                good += int(dom.select('button._0mzm-.sqdOP.yWX7d._8A5w5 > span')[0].text.replace(',', ''))

                if len(dom.select('div.D1AKJ > a')) == 1 and dom.select('div.D1AKJ > a')[0].text == '이전':    
                    #account_info['처음활동날짜'] = dom.select('time._1o9PC.Nzb55')[0]['title']
                    first_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10])

                    first_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0])
                    first_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1])
                    first_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2])

                    last = date(last_Year, last_Month, last_Day)
                    first = date(first_Year, first_Month, first_Day)

                    period = last - first
                    active_Period.append(period.days)

                    comment_Count.append(comment)
                    good_Count.append(good)
                    avg_Comment.append(round(comment / post_Count[0]))
                    avg_Good.append(round(good / post_Count[0]))

                    words = [j.text for j in hashtags]

                    hashtags_data = [x for x in words if x[0] == '#']

                    hashtags_Count.append(Counter(hashtags_data).most_common()[:1])

                    #total_data = {'계정ID':account_Id, '총게시물수':post_Count, '팔로워수':follower_Count, '평균댓글':avg_Comment, '평균좋아요':avg_Good, '총댓글수':comment_Count, '총좋아요수':good_Count, '최근게시물날짜':last_Activity_Date, '처음게시물날짜':first_Activity_Date, '활성기간':active_Period, '해시태그/빈도수':hashtags_Count}

                    #df = pd.DataFrame(total_data)
                    #df.to_csv('/Users/markmac/Downloads/instagram_accounts_data.csv')

                    driver.close()
                    break

                else:
                    driver.find_element_by_xpath("//a[@class = 'HBoOv coreSpriteRightPaginationArrow']").click()
                    time.sleep(1)

            # 좋아요 수가 0일때 사진게시물
            elif dom.select('div.Igw0E.IwRSH.eGOV_.ybXk5.vwCYk') != [] and good == 0 and comment != 0:
                comment += len(dom.select('li.gElp9'))

                hashtags += dom.select('div.C4VMK > span > a')

                good = int(dom.select('button._0mzm-.sqdOP.yWX7d._8A5w5 > span')[0].text.replace(',', ''))

                if len(dom.select('div.D1AKJ > a')) == 1 and dom.select('div.D1AKJ > a')[0].text == '이전':    
                    #account_info['처음활동날짜'] = dom.select('time._1o9PC.Nzb55')[0]['title']
                    first_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10])

                    first_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0])
                    first_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1])
                    first_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2])

                    last = date(last_Year, last_Month, last_Day)
                    first = date(first_Year, first_Month, first_Day)

                    period = last - first
                    active_Period.append(period.days)

                    comment_Count.append(comment)
                    good_Count.append(good)
                    avg_Comment.append(round(comment / post_Count[0]))
                    avg_Good.append(round(good / post_Count[0]))

                    words = [j.text for j in hashtags]

                    hashtags_data = [x for x in words if x[0] == '#']

                    hashtags_Count.append(Counter(hashtags_data).most_common()[:1])

                    #total_data = {'계정ID':account_Id, '총게시물수':post_Count, '팔로워수':follower_Count, '평균댓글':avg_Comment, '평균좋아요':avg_Good, '총댓글수':comment_Count, '총좋아요수':good_Count, '최근게시물날짜':last_Activity_Date, '처음게시물날짜':first_Activity_Date, '활성기간':active_Period, '해시태그/빈도수':hashtags_Count}

                    #df = pd.DataFrame(total_data)
                    #df.to_csv('/Users/markmac/Downloads/instagram_accounts_data.csv')

                    driver.close()
                    break

                else:
                    driver.find_element_by_xpath("//a[@class = 'HBoOv coreSpriteRightPaginationArrow']").click()
                    time.sleep(1)

            # 둘다 0일때 사진 게시물
            else:
                comment += len(dom.select('li.gElp9'))

                hashtags += dom.select('div.C4VMK > span > a')

                good += int(dom.select('button._0mzm-.sqdOP.yWX7d._8A5w5 > span')[0].text.replace(',', ''))

                if len(dom.select('div.D1AKJ > a')) == 1 and dom.select('div.D1AKJ > a')[0].text == '이전':    
                    #account_info['처음활동날짜'] = dom.select('time._1o9PC.Nzb55')[0]['title']
                    first_Activity_Date.append(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10])

                    first_Year = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[0])
                    first_Month = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[1])
                    first_Day = int(dom.select('time._1o9PC.Nzb55')[0]['datetime'][:10].split('-')[2])

                    last = date(last_Year, last_Month, last_Day)
                    first = date(first_Year, first_Month, first_Day)

                    period = last - first
                    active_Period.append(period.days)

                    comment_Count.append(comment)
                    good_Count.append(good)
                    avg_Comment.append(round(comment / post_Count[0]))
                    avg_Good.append(round(good / post_Count[0]))

                    words = [j.text for j in hashtags]

                    hashtags_data = [x for x in words if x[0] == '#']

                    hashtags_Count.append(Counter(hashtags_data).most_common()[:1])

                    #total_data = {'계정ID':account_Id, '총게시물수':post_Count, '팔로워수':follower_Count, '평균댓글':avg_Comment, '평균좋아요':avg_Good, '총댓글수':comment_Count, '총좋아요수':good_Count, '최근게시물날짜':last_Activity_Date, '처음게시물날짜':first_Activity_Date, '활성기간':active_Period, '해시태그/빈도수':hashtags_Count}

                    #df = pd.DataFrame(total_data)
                    #df.to_csv('/Users/markmac/Downloads/instagram_accounts_data.csv')

                    driver.close()
                    break

                else:
                    driver.find_element_by_xpath("//a[@class = 'HBoOv coreSpriteRightPaginationArrow']").click()
                    time.sleep(1)

        # 댓글 더보기 버튼이 있으면
        else:
            #more_Comment_Wait = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'button.Z4IfV._0mzm-.sqdOP.yWX7d')))
            #more_Comment_Wait.click()

            # 댓글 더 보기 버튼 클릭
            driver.find_element_by_xpath("//button[@class = 'Z4IfV _0mzm- sqdOP yWX7d        ']").click()
            time.sleep(2)
    except:
        break
# dict 형식으로 모든 데이터 저장
total_data = {'계정ID':account_Id, '총게시물수':post_Count, '팔로워수':follower_Count, '평균댓글':avg_Comment, '평균좋아요':avg_Good, '총댓글수':comment_Count, '총좋아요수':good_Count, '최근게시물날짜':last_Activity_Date, '처음게시물날짜':first_Activity_Date, '활성기간':active_Period, '해시태그/빈도수':hashtags_Count}

#output.put(total_data)

df = pd.DataFrame(total_data) # pandas dataframe 형식으로 저장
df.to_csv('/Users/markmac/Downloads/instagram_accounts_data.csv') # csv 파일로 저장

ValueError: arrays must all be same length